### Q1

In [1]:
import networkx as nx
import numpy as np
import matlab.engine

# Helper function to load and analyze a graph
def analyze_graph(file_path, network_name):
    # Step 1: Load the undirected graph using networkx
    G_nx = nx.read_edgelist(file_path, create_using=nx.Graph(), nodetype=int)

    # Step 2: Check the basic properties of the graph
    num_nodes = G_nx.number_of_nodes()
    num_edges = G_nx.number_of_edges()
    
    # Degree statistics
    degrees = np.array([deg for (node, deg) in G_nx.degree()])
    max_degree = np.max(degrees)
    min_degree = np.min(degrees)
    avg_degree = np.mean(degrees)

    # Degree distribution type (simplified: scale-free or unknown)
    if np.max(degrees) > 10 * np.mean(degrees):  # rough heuristic for scale-free
        degree_dist_type = 'Scale-free'
    else:
        degree_dist_type = 'Unknown'

    # Step 3: Check for connectivity (undirected graph)
    is_connected = nx.is_connected(G_nx)
    if not is_connected:
        diameter = 'N/A - Graph not connected'
    else:
        diameter = 'Good'  # Diameter only makes sense for connected graphs
        

    # Print out the required information
    print(f"--- {network_name} ---")
    print(f"n (Number of nodes): {num_nodes}")
    print(f"m (Number of edges): {num_edges}")
    print(f"min(d) (Minimum degree): {min_degree}")
    print(f"max(d) (Maximum degree): {max_degree}")
    print(f"avg(d) (Average degree): {avg_degree:.2f}")
    print(f"Degree distribution type: {degree_dist_type}")
    print(f"Diameter: {diameter}")
    print()

    # Step 4: Compute the Local Clustering Coefficient (LCC) for each node
    lcc_dict = nx.clustering(G_nx)
    lcc_values = np.array(list(lcc_dict.values()))  # Convert to numpy array

    return degrees, lcc_values

# Helper function to plot distributions using MATLAB
def plot_with_matlab(eng, degree, lcc_values, network_name):
    nbins = 50
    
    # Plot Degree Distribution
    eng.workspace['d'] = degree
    figID = 1
    eng.eval(f"plot_distribution(d, '{network_name} Degree Distribution', {nbins}, {figID})", nargout=0)
    
    # Plot LCC Distribution
    eng.workspace['lcc'] = lcc_values
    figID = 2
    eng.eval(f"plot_distribution(lcc, '{network_name} LCC Distribution', {nbins}, {figID})", nargout=0)


In [7]:

eng = matlab.engine.start_matlab()
eng.eval("addpath(genpath('Mcodes'))", nargout=0)

file_path_amazon = 'data/com-amazon.ungraph/com-amazon.ungraph.txt'
degree_amazon, lcc_amazon = analyze_graph(file_path_amazon, "Amazon")
plot_with_matlab(eng, degree_amazon, lcc_amazon, "Amazon")

--- Amazon ---
n (Number of nodes): 334863
m (Number of edges): 925872
min(d) (Minimum degree): 1
max(d) (Maximum degree): 549
avg(d) (Average degree): 5.53
Degree distribution type: Scale-free
Diameter: Good



In [3]:

file_path_facebook = 'data/facebook/facebook_combined.txt'
degree_facebook, lcc_facebook = analyze_graph(file_path_facebook, "Facebook")
plot_with_matlab(eng, degree_facebook, lcc_facebook, "Facebook")

--- Facebook ---
n (Number of nodes): 4039
m (Number of edges): 88234
min(d) (Minimum degree): 1
max(d) (Maximum degree): 1045
avg(d) (Average degree): 43.69
Degree distribution type: Scale-free
Diameter: Good



In [4]:

file_path_dblp = 'data/COM-DBLP/com-dblp.ungraph.txt'
degree_dblp, lcc_dblp = analyze_graph(file_path_dblp, "DBLP")
plot_with_matlab(eng, degree_dblp, lcc_dblp, "DBLP")



--- DBLP ---
n (Number of nodes): 317080
m (Number of edges): 1049866
min(d) (Minimum degree): 1
max(d) (Maximum degree): 343
avg(d) (Average degree): 6.62
Degree distribution type: Scale-free
Diameter: Good



In [5]:
eng.exit()

### Q2

In [2]:
# Helper function to load G_1


def load_g1(file_path):
    # Load the undirected graph using networkx
    G_nx = nx.read_edgelist(file_path, create_using=nx.Graph(), nodetype=int)
    return G_nx


# New function to compute the G_2 graph from G_1
def construct_g2_sparse(G_nx):
    # Compute the sparse adjacency matrix of G_1
    A1_sparse = nx.to_scipy_sparse_array(G_nx)  # This is the correct method
    
    # Compute A_2 = A_1^2 (2-step connectivity) using sparse matrix multiplication
    A2_sparse = A1_sparse.dot(A1_sparse)

    # Create the G_2 graph from the sparse matrix
    G2_nx = nx.from_scipy_sparse_array(A2_sparse)

    return G2_nx


# Helper function to analyze a graph (G_2)
def analyze_g2(G2_nx, network_name):
    # Degree statistics
    degrees = np.array([deg for (node, deg) in G2_nx.degree()])
    max_degree = np.max(degrees)
    min_degree = np.min(degrees)
    avg_degree = np.mean(degrees)

    # # Degree distribution type (simplified: scale-free or unknown)
    # if np.max(degrees) > 10 * np.mean(degrees):  # rough heuristic for scale-free
    #     degree_dist_type = 'Scale-free'
    # else:
    #     degree_dist_type = 'Unknown'

    # Step 3: Check for connectivity (undirected graph)
    # is_connected = nx.is_connected(G2_nx)
    # diameter = nx.diameter(G2_nx) if is_connected else 'N/A - Graph not connected'

    # Print out the required information
    print(f"--- {network_name} (G2) ---")
    print(f"min(d) (Minimum degree): {min_degree}")
    print(f"max(d) (Maximum degree): {max_degree}")
    print(f"avg(d) (Average degree): {avg_degree:.2f}")
    # print(f"Degree distribution type: {degree_dist_type}")
    # print(f"Diameter: {diameter}")
    print()

    # Step 4: Compute the Local Clustering Coefficient (LCC) for each node
    lcc_dict = nx.clustering(G2_nx)
    lcc_values = np.array(list(lcc_dict.values()))  # Convert to numpy array

    return degrees, lcc_values


# Helper function to plot distributions using MATLAB
def plot_with_matlab(eng, degree, lcc_values, network_name, graph_label="G2"):
    nbins = 50

    # Plot Degree Distribution
    eng.workspace['d'] = degree
    figID = 1
    eng.eval(f"plot_distribution(d, '{network_name} Degree Distribution ({graph_label})', {nbins}, {figID})", nargout=0)

    # Plot LCC Distribution
    eng.workspace['lcc'] = lcc_values
    figID = 2
    eng.eval(f"plot_distribution(lcc, '{network_name} LCC Distribution ({graph_label})', {nbins}, {figID})", nargout=0)


eng = matlab.engine.start_matlab()
eng.eval("addpath(genpath('Mcodes'))", nargout=0)

In [7]:

file_path_amazon = 'data/com-amazon.ungraph/com-amazon.ungraph.txt'
G1_amazon = load_g1(file_path_amazon)  # Load G1 (recompute)

G2_amazon = construct_g2_sparse(G1_amazon)  # Construct G2 from G1
degree_amazon_g2, lcc_amazon_g2 = analyze_g2(G2_amazon, "Amazon")  # Analyze G2

# Plot results for G2 (Amazon)
plot_with_matlab(eng, degree_amazon_g2, lcc_amazon_g2, "Amazon", "G2")

--- Amazon (G2) ---
min(d) (Minimum degree): 3
max(d) (Maximum degree): 1328
avg(d) (Average degree): 42.15



In [6]:

file_path_facebook = 'data/facebook/facebook_combined.txt'
G1_facebook = load_g1(file_path_facebook)  # Load G1 (recompute)

G2_facebook = construct_g2_sparse(G1_facebook)  # Construct G2 from G1
degree_facebook_g2, lcc_facebook_g2 = analyze_g2(G2_facebook, "Facebook")  # Analyze G2

# Plot results for G2 (Facebook)
plot_with_matlab(eng, degree_facebook_g2, lcc_facebook_g2, "Facebook", "G2")

--- Facebook (G2) ---
min(d) (Minimum degree): 58
max(d) (Maximum degree): 2916
avg(d) (Average degree): 718.13



In [3]:

file_path_dblp = 'data/COM-DBLP/com-dblp.ungraph.txt'
G1_dblp = load_g1(file_path_dblp)  # Load G1 (recompute)

G2_dblp = construct_g2_sparse(G1_dblp)  # Construct G2 from G1
degree_dblp_g2, lcc_dblp_g2 = analyze_g2(G2_dblp, "DBLP")  # Analyze G2

# Plot results for G2 (DBLP)
plot_with_matlab(eng, degree_dblp_g2, lcc_dblp_g2, "DBLP", "G2")

--- DBLP (G2) ---
min(d) (Minimum degree): 3
max(d) (Maximum degree): 5396
avg(d) (Average degree): 88.07



In [ ]:

eng.exit()